In [1]:
# Generate data to infer the rules for integral

import json
import numpy as np
import sympy as sp
from sympy import sympify, lambdify, symbols, integrate, Interval, Symbol, I, S, oo, plot, evalf, N
from IPython.display import display
from utils.utils import *


# def remove_constants(f):
#     t = Symbol('t')
#     return f.as_independent(t)[1]

# fin = open("datasets/parametric_equations_polynomial_integral_results.json", "r")
# lines = fin.readlines()
# fin.close()
# fin = open("datasets/parametric_equations_randomized_polynomial_integral_results.json", "r")
# lines.extend(fin.readlines())
# fin.close()

# MAX_POWER = 6
# MAX_AVG_DIFF = 0.01

# originals = []
# integrals = []
# t = Symbol('t')

# for line in lines:
#     result = json.loads(line)
#     if "rounded_regressed" not in result:
#         continue
#     original = round_all_floats(N(sympify(result["original"])))
#     integral = remove_constants(round_all_floats(N(sympify(result["rounded_regressed"]))))
#     try:
#         original = filter_non_polynomial(original)
#         integral = filter_non_polynomial(integral)
#         original_integral = integrate(original, t)
#         avg_diff = get_avg_diff(original_integral, integral, t)
#         if avg_diff > MAX_AVG_DIFF or len(original_integral.args) != len(integral.args):
#             print("Skipping. Diff=", avg_diff)
#             display(original_integral)
#             display(integral)
#             continue
#     except:
#         print("Cannot filter non-polynomials on", str(integral))
#         continue
#     originals.append(str(original))
#     integrals.append(str(integral))
#     if len(originals) % 100 == 0:
#         print(len(originals), "cases loaded")
    
# fin.close()

# sentences = []
# for i in range(len(originals)):
#     sentences.append(originals[i] + ' entail ' + integrals[i] + ' end')
# fin.close()



In [2]:
import json
import torch
import random
from datasets import load_dataset, Dataset, load_from_disk

# random.shuffle(sentences)

# ds = Dataset.from_dict({'eq_pair': sentences})
# train_ds = ds.train_test_split(test_size=0.025)

# train_ds['train'][1]

# train_ds.save_to_disk("datasets/integrate_gptneo_dataset_50k")

train_ds = load_from_disk("datasets/integrate_gptneo_dataset_50k")

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_ds

DatasetDict({
    train: Dataset({
        features: ['eq_pair'],
        num_rows: 47881
    })
    test: Dataset({
        features: ['eq_pair'],
        num_rows: 1228
    })
})

In [4]:
from transformers import AutoTokenizer

CONTEXT_LENGTH = 256

tokenizer = AutoTokenizer.from_pretrained("xhyi/PT_GPTNEO350_ATG") #"EleutherAI/gpt-neo-125m") 
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    return tokenizer(examples["eq_pair"], padding='max_length', truncation=True, max_length=CONTEXT_LENGTH, return_tensors="pt")

tokenized_ds = train_ds.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=train_ds["train"].column_names,
)

def preprocess_function(examples):
    examples["labels"] = examples["input_ids"].copy()
    return examples

lm_dataset = tokenized_ds.map(preprocess_function, batched=True, num_proc=1)

from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [5]:
# from transformers import AutoTokenizer

# CONTEXT_LENGTH = 256

# tokenizer = AutoTokenizer.from_pretrained("xhyi/PT_GPTNEO350_ATG")

# tokens = tokenizer.encode("0.1428571428571")
# for token in tokens:
#     print(tokenizer.decode([token]))


In [6]:
# for i in range(len(lm_dataset['train'])):
#     if len(lm_dataset['train'][i]['input_ids']) != 256:
#         print(i, len(lm_dataset['train'][i]['input_ids']))
#     if len(lm_dataset['train'][i]['labels']) != 256:
#         print(i, len(lm_dataset['train'][i]['labels']))

In [5]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("datasets/amps_mathematica_algebra_numtheory_gptneo_350m/gptneo-350m-985000.model")  #"EleutherAI/gpt-neo-125m")
# config = model0.config

# model = AutoModelForCausalLM.from_config(config)
# model0 = None

In [7]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="datasets/integrate_gptneo_350m_w_pretrain_50k_202401",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    fp16=True,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=1000,
    save_total_limit=20,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss,Validation Loss
1000,0.765100,0.570765
2000,0.535600,0.513106
3000,0.495000,0.484706
4000,0.464000,0.471766
5000,0.448700,0.456800
6000,0.439800,0.447679
7000,0.407100,0.438985
8000,0.404000,0.431053
9000,0.401500,0.425800
10000,0.365500,0.425145


Checkpoint destination directory datasets/integrate_gptneo_350m_w_pretrain_50k_202401/checkpoint-13000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory datasets/integrate_gptneo_350m_w_pretrain_50k_202401/checkpoint-14000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=14965, training_loss=0.42853959246379575, metrics={'train_runtime': 6119.8915, 'train_samples_per_second': 39.119, 'train_steps_per_second': 2.445, 'total_flos': 1.1114068317831168e+17, 'train_loss': 0.42853959246379575, 'epoch': 5.0})

In [9]:
model.save_pretrained("datasets/integrate_gptneo_350m_w_pretrain_50k_202401/gptneo-350m-14500.model")

In [10]:
# device = "cuda:0" if torch.cuda.is_available() else "cpu"

# model = AutoModelForCausalLM.from_pretrained("datasets/integrate_gptneo_202401/gptneo-350m-5500-loss0.350.model").to(device)

In [6]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

def generate_integral(input):
    # Encode some input text
    prompt = input + " entail"
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    # Generate text
    output = model.generate(input_ids, max_length=100, num_return_sequences=1, temperature=0.7)
    # Decode and print the output
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    #print(generated_text)
    if 'entail' in generated_text:
        generated_text = generated_text[generated_text.find('entail') + 6:].strip()
        #print(generated_text)
    if 'end' in generated_text:
        generated_text = generated_text[0:generated_text.find('end')].strip()
    return generated_text
    
print(generate_integral("2.5*t**3 + 0.51*t**2 + 3.2*t + 1.2"))

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/transformers/generation/utils.py:1636: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by ca

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [9]:
train_ds['test'][0]

{'eq_pair': '14.0*t**4 - 5.87*t + 13.44 entail 2.8*t**5 - 2.94*t**2 + 13.44*t end'}

In [11]:
from transformers import TrainingArguments, Trainer
import sympy as sp
from utils.utils import *
import time

MIN_ALLOWED_DIFF = 0.011

test_ds = train_ds['test']

training_args = TrainingArguments(
    output_dir="datasets/integrate_gptneo_350m_w_pretrain_50k_202401",
    evaluation_strategy="steps",
    learning_rate=2e-7,
    weight_decay=0.01,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=0.001,
    fp16=True,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=1000,
    save_total_limit=20,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

for step in range(12000, 12500, 1000):
    print("Recovering checkpoint", step)
    cp_path = "datasets/integrate_gptneo_350m_w_pretrain_50k_202401/checkpoint-" + str(step)
    trainer.train(cp_path)
    #
    num_processed = 0
    num_equal = 0
    num_zero_diff = 0
    num_within_allowed = 0
    total_time = 0.0
    t = sp.Symbol('t')
    #
    verbose = False
    #
    for i in range(min(len(test_ds), 1000)):
        if verbose:
            print("Case", i, test_ds[i]['eq_pair'])
        eq_pair = test_ds[i]['eq_pair']
        question = eq_pair[0:eq_pair.find('entail')].strip()
        answer = eq_pair[eq_pair.find('entail') + 6:].replace('end', '').strip()
        if 'repeat' in question:
            question = question[0:question.find('repeat')]
        original = sp.sympify(question)
        integral = round_all_floats(sp.integrate(original), 2)
        if verbose: display(integral)
        try:
            t1 = time.time()
            pred = generate_integral(question)
            generated = round_all_floats(sp.sympify(pred), 2)
            t2 = time.time()
            total_time += (t2-t1)
            if verbose: display(generated)
        except:
            print("Cannot sympify", pred)
            continue
        avg_diff = get_avg_diff(integral, generated, t)
        if verbose: print("avg_diff", avg_diff)
        num_processed += 1
        if avg_diff <= MIN_ALLOWED_DIFF:
            num_within_allowed += 1
        else:
            display(integral)
            display(generated)
            print(avg_diff)
        if avg_diff <= 0.0000001:
            num_zero_diff += 1
        diff_expr = sp.simplify(integral-generated)
        if verbose: display(diff_expr)
        if diff_expr == 0:
            if verbose: print("Equal")
            num_equal += 1
        if i%100 == 0:
            print(i+1, "processed")
    #
    print("num_processed", num_processed)
    print("num_equal", num_equal)
    print("num_zero_diff", num_zero_diff)
    print("num_within_allowed", num_within_allowed)
    print("avg. time =", total_time/num_processed)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You are resuming training from a checkpoint trained with 4.36.1 of Transformers but your current version is 4.36.2. This is not recommended and could yield to errors or unwanted behaviors.


Recovering checkpoint 12000


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Step,Training Loss,Validation Loss


/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

0.8*t**5 + 3.88*t**3 + 4.16*t**2 + 20.83*t

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.015911914964079794


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

num_processed 1000
num_equal 733
num_zero_diff 733
num_within_allowed 953
avg. time = 0.695363799571991


In [1]:
# Load symbolicregression model

import torch
import os, sys
import symbolicregression
import sympytorch
import requests
from sympy.core.rules import Transform

model_path = "ckpt/model.pt" 
try:
    if not os.path.isfile(model_path): 
        print("Downloading model...")
        url = "https://dl.fbaipublicfiles.com/symbolicregression/model1.pt"
        r = requests.get(url, allow_redirects=True)
        open(model_path, 'wb').write(r.content)
    if not torch.cuda.is_available():
        sr_model = torch.load(model_path, map_location=torch.device('cpu'))
    else:
        sr_model = torch.load(model_path)
        sr_model = sr_model.cuda()
    print(sr_model.device)
    print("Model successfully loaded!")

except Exception as e:
    print("ERROR: model not loaded! path was: {}".format(model_path))
    print(e)    
    
est = symbolicregression.model.SymbolicTransformerRegressor(
                        model=sr_model,
                        max_input_points=10001,
                        n_trees_to_refine=5,
                        rescale=True
                        )

import numpy as np
import sympy as sp
from sympy import sympify, lambdify, symbols, integrate, Interval, Symbol, I, S, oo, plot
from IPython.display import display

# Given an expr f (of variable t), returns its integral, together with t's and y's for regression
def integrate_expr(f, min_x=-4.0, max_x=4.0, increment=0.002, verbose=False):
    if verbose:
        print("Running integration on")
        display(f)
    # Compute integration
    x, t = symbols(['x','t'])
    fi = integrate(f, t)
    if verbose:
        display(fi)
        #plot(fi, (t, min_x, max_x))
    # Generate data for symbolic regression
    fl = lambdify((t), fi, "numpy")
    ts = np.arange(min_x, max_x, increment)
    ys = fl(ts)
    return fi, ts, ys
    
integrate_expr(sympify("2*t**2-t*2+2.5"), verbose=True)

cuda:0
Model successfully loaded!
Running integration on


2*t**2 - 2*t + 2.5

0.666666666666667*t**3 - 1.0*t**2 + 2.5*t

(0.666666666666667*t**3 - 1.0*t**2 + 2.5*t,
 array([-4.   , -3.998, -3.996, ...,  3.994,  3.996,  3.998]),
 array([-68.66666667, -68.58170266, -68.49681062, ...,  36.50791852,
         36.56077862,  36.61369466]))

In [2]:
from utils.utils import *

def round_expr(expr, num_digits=2):
    return expr.xreplace(Transform(lambda x: x.round(num_digits), lambda x: isinstance(x, sp.Float)))

# Run symbolic regression on given data
# Returns: (raw regressed expr, rounded expr, model refined expr)
@timeout(15)
def symbolic_regress(sr_model, xs, ys, generate_refinement=False, verbose=False):
    if verbose:
        print("Running Symbolic Regression...")
    ##Example of data
    xs = np.reshape(xs, (len(xs),1))
    ys = np.reshape(ys, (len(ys),1))
    sr_model.fit(xs,ys)
    #
    replace_ops = {"add": "+", "mul": "*", "sub": "-", "pow": "**", "inv": "1/"}
    model_str = sr_model.retrieve_tree(with_infos=True)["relabed_predicted_tree"].infix()
    for op,replace_op in replace_ops.items():
        model_str = model_str.replace(op,replace_op)
    #
    raw_expr = sp.parse_expr(model_str)
    x_0, t = symbols(['x_0', 't'])
    raw_expr = raw_expr.subs(x_0, t)
    if verbose:
        display(raw_expr)
    #
    expr = sp.expand(raw_expr)
    rounded_expr = round_expr(expr)
    if verbose:
        display(rounded_expr)
    #
    # Encode some input text
    if generate_refinement:
        prompt = str(rounded_expr)
        input_ids = tokenizer.encode(prompt, return_tensors='pt')
        #
        # Generate text
        output = model.generate(input_ids, max_length=50, num_return_sequences=1, temperature=0.1)
        #
        # Decode and print the output
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_expr = sympify(generated_text)
        if verbose:
            display(generated_expr)
        #
    else:
        generated_expr = None
    return raw_expr, rounded_expr, generated_expr


fi, ts, ys = integrate_expr(sympify("2*t**2-t*2+2.5"), verbose=False)
rounded_fi = round_expr(fi)
display(rounded_fi)
raw_expr, rounded_expr, generated_expr = symbolic_regress(est, ts, ys, verbose=False)
display(rounded_expr)

print("Diff1:")
display(rounded_fi-rounded_expr)

0.67*t**3 - 1.0*t**2 + 2.5*t

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')


0.67*t**3 - 1.0*t**2 + 2.5*t

Diff1:


0

In [3]:
import json
import numpy as np
import sympy as sp
from sympy import sympify, lambdify, symbols, integrate, Interval, Symbol, I, S, oo, plot, evalf, N
from IPython.display import display
from utils.utils import *
import time
import random

MIN_ALLOWED_DIFF = 0.011
verbose = False
t = sp.Symbol('t')

fin = open("datasets/parametric_equations_randomized_polynomial_integral_results.json", "r")
lines = fin.readlines()
fin.close()

random.seed(1234)
random.shuffle(lines)

num_seen = 0
num_processed = 0
num_equal = 0
num_zero_diff = 0
num_within_allowed = 0
total_time = 0.0

for i in range(2000):
    line = lines[i]
    result = json.loads(line)
    if "rounded_regressed" not in result:
        continue
    original_expr = round_all_floats(N(sympify(result["original"])))
    if verbose:
        print("Case", i, original)
    try:
        original = sp.sympify(original_expr)
        integral = round_all_floats(sp.integrate(original), 2)
    except:
        print("Cannot initialize", original_expr)
        continue
    num_seen += 1
    if verbose: display(integral)
    try:
        t1 = time.time()
        fi, ts, ys = integrate_expr(original, verbose=False)
        raw_expr, generated, _ = symbolic_regress(est, ts, ys, verbose=False)
        t2 = time.time()
        total_time += (t2-t1)
    except:
        print("Cannot process", original_expr)
        continue
    avg_diff = get_avg_diff(integral, generated, t)
    if verbose: print("avg_diff", avg_diff)
    num_processed += 1
    if avg_diff <= MIN_ALLOWED_DIFF:
        num_within_allowed += 1
    else:
        display(integral)
        display(generated)
        print(avg_diff)
    if avg_diff <= 0.0000001:
        num_zero_diff += 1
    try:
        diff_expr = sp.simplify(integral-generated)
        if verbose: display(diff_expr)
        if diff_expr == 0:
            #if verbose: 
            print("Equal")
            num_equal += 1
    except:
        print("Cannot process diff_expr")
        continue
    if num_seen%100 == 0:
        print(num_seen, "checked")
    if num_seen >= 1000:
        break
#
print("num_seen", num_seen)
print("num_processed", num_processed)
print("num_equal", num_equal)
print("num_zero_diff", num_zero_diff)
print("num_within_allowed", num_within_allowed)
print("avg. time =", total_time/num_processed)

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')
/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('gra

Equal


/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')


Equal


/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')


...

Equal


/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')


Equal


/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/torch/_functorch/deprecated.py:62: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.grad is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.func.grad instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('grad')


Equal
1000 checked
num_seen 1000
num_processed 999
num_equal 656
num_zero_diff 657
num_within_allowed 906
avg. time = 3.444702995670689
